In [1]:
import numpy as np
import pandas as pd
import time
import wrds
from matplotlib import pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
from scipy.interpolate import UnivariateSpline, splev, splrep
from scipy.optimize import minimize
from math import *
np.random.seed(6078)
import warnings
warnings.filterwarnings('ignore')
# from dask.multiprocessing import get
# import dask.dataframe as dd

In [2]:
opm = pd.read_csv('merged_data.csv', nrows=200000)
#opm['date'] = pd.to_datetime(opm['date'], format='%Y-%m-%d')
#opm['exdate'] = pd.to_datetime(opm['exdate'], format='%Y-%m-%d')
# opm = opm[(opm['exdate'] - opm['date'] == np.timedelta64(-8, 'D'))]
opm.rename(columns={'strike_price': 'STRIKE'}, inplace=True)
opm['STRIKE'] = opm['STRIKE']/1000

In [3]:
opm.head()

,secid,exdate,date,symbol,cp_flag,STRIKE,best_bid,best_offer,volume,open_interest,optionid,forward_price,index_flag,issuer,exercise_style,year
0,210354,20180119,20180112,AA 180119C17000,C,17.0,39.6,39.8,0,0,115694181,NaN,0,ALCOA CORP,A,2018
1,210354,20180119,20180112,AA 180119C18000,C,18.0,38.5,38.8,0,4,115145533,NaN,0,ALCOA CORP,A,2018
2,210354,20180119,20180112,AA 180119C19000,C,19.0,37.6,37.9,0,2,115694182,NaN,0,ALCOA CORP,A,2018
3,210354,20180119,20180112,AA 180119C20000,C,20.0,36.6,36.9,0,99,114695511,NaN,0,ALCOA CORP,A,2018
4,210354,20180119,20180112,AA 180119C21000,C,21.0,35.6,35.8,0,12,115694183,NaN,0,ALCOA CORP,A,2018


In [9]:
def clean_fairs(chain):
    chain = chain.sort_values(["cp_flag", "STRIKE"]).reset_index()
    phain = chain[chain['cp_flag'] == 'P']
    chain = chain[chain['cp_flag'] == 'C']
    chain['best_bid'] = -((-chain['best_bid']).cummax())
    chain['best_offer'] = -((-chain['best_offer']).cummax())
    phain['best_bid'] = ((phain['best_bid']).cummax())
    phain['best_offer'] = ((phain['best_offer']).cummax())
    chain["CV"] = (chain["best_bid"] + chain["best_offer"]) / 2
    phain["PV"] = (phain["best_bid"] + phain["best_offer"]) / 2
    chain = chain[["STRIKE", "CV"]]
    phain = phain[["STRIKE", "PV"]]
    df = chain.merge(phain, on='STRIKE')
    df["V"] = df["CV"] + df["PV"]# - (df['STRIKE'] - prc).abs()
    return df

In [117]:
# def clean_fairs(chain):
#     chain = chain.sort_values(["cp_flag", "STRIKE"]).reset_index()
#     phain = chain[chain['cp_flag'] == 'P']
#     chain = chain[chain['cp_flag'] == 'C']
#     chain['CV'] = -((-chain['opprc']).cummax())
#     phain['PV'] = ((phain['opprc']).cummax())
#     chain = chain[["STRIKE", "CV"]]
#     phain = phain[["STRIKE", "PV"]]
#     df = chain.merge(phain, on='STRIKE')
#     df["V"] = df["CV"] + df["PV"]
#     return df

In [10]:
def implied_probs(chain, s=0.1, M=1000):
    df = clean_fairs(chain)
    df =  df.drop_duplicates(subset='STRIKE', keep="first").reset_index()
    if len(df) < 5:
        return np.nan
    spl = UnivariateSpline(df['STRIKE'], df['V'], k=4, s=s*df.shape[0])
    df['Vhat'] = spl(df['STRIKE'])
    df['IP'] = np.maximum(spl.derivative().derivative()(df['STRIKE']), 0)
    df["PCDF"] = df["IP"].cumsum()
    df["PCDF"] = df["PCDF"] / df["PCDF"].max()
    return df

In [11]:
def implied_median(chain):
    df = implied_probs(chain)
    try:
        mid_idx = df[(df['PCDF'] > 0.5)].index[0]
        lb = df.loc[mid_idx-1, 'STRIKE']
        lbp = df.loc[mid_idx-1, 'PCDF']
        ub = df.loc[mid_idx, 'STRIKE']
    except:
        return np.nan
    
    ubp = df.loc[mid_idx, 'PCDF']
    slope = (ubp - lbp) / (ub - lb)
    res = (0.5 - lbp) / slope + lb
    return res

def implied_move(chain):
    return implied_median(chain) / chain.PRC.max() - 1

In [120]:
# chain = opm[opm['secid'] == 106367]
# chain = chain[chain['exdate'] == '2018-01-15']
# df = implied_probs(chain)
# print(implied_median(chain))

In [7]:
opm.columns

Index(['secid', 'exdate', 'date', 'symbol', 'cp_flag', 'STRIKE', 'best_bid',
       'best_offer', 'volume', 'open_interest', 'optionid', 'forward_price',
       'index_flag', 'issuer', 'exercise_style', 'year'],
      dtype='object')

In [122]:
# df.plot(x='STRIKE', y='Vhat')
# df.plot(x='STRIKE', y='IP')
# df.plot(x='STRIKE', y='PCDF')

In [123]:
(df['IP'] * df['STRIKE']).sum() / df['IP'].sum()

372.30307126506295

In [12]:
opm.groupby(["secid", "exdate"]).apply(implied_median)

secid   exdate  
8170    20180615     20.172486
100862  20181221     38.995916
100863  20180420     48.151437
        20181019     41.366786
100885  20181221     21.659527
                       ...    
212968  20181116     22.295341
213004  20180817           NaN
        20181116           NaN
213114  20181116    101.881479
213142  20181116     12.820376
Length: 1442, dtype: float64

In [15]:
opm_app_mean = opm.groupby(["secid", "exdate"]).apply(implied_median)

In [16]:
opm_app_mean.to_csv("opm_app_mean.csv")